In [ ]:
"""
STATISTIQUES DESCRIPTIVES - IMPACT DES ZFE SUR LE NO₂
Projet Python pour la Data Science - Analyse causale Grenoble & Paris
"""

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import json
from shapely.geometry import Point, shape
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings('ignore')

# Configuration graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

# ============================================================================
# 0. CHEMINS ET CHARGEMENT DES DONNÉES
# ============================================================================

HERE = Path().resolve()
ROOT = HERE.parent if 'zfe-scm' in str(HERE) else HERE
DATA = ROOT / "data"

print("=" * 80)
print("📊 STATISTIQUES DESCRIPTIVES - IMPACT DES ZFE SUR LE NO₂")
print("=" * 80)

# Charger les métadonnées ZFE
zfe_meta = pd.read_csv(DATA / "zfe_meta.csv")
zfe_meta['first_date_debut'] = pd.to_datetime(zfe_meta['first_date_debut'])

# Dates de début des ZFE
grenoble_zfe_start = zfe_meta.loc[zfe_meta['publisher_zfe_id'] == 'GRENOBLE', 'first_date_debut'].iloc[0]
paris_zfe_start = zfe_meta.loc[zfe_meta['publisher_zfe_id'] == 'PARIS', 'first_date_debut'].iloc[0]

print(f"\n🎯 Dates clés :")
print(f"  • ZFE Grenoble : {grenoble_zfe_start.date()}")
print(f"  • ZFE Paris : {paris_zfe_start.date()}")

# Charger les données de pollution
grenoble_daily = pd.read_csv(DATA / "pollution_grenoble_no2_daily_clean.csv")
grenoble_daily['date'] = pd.to_datetime(grenoble_daily['date'])

paris_daily = pd.read_csv(DATA / "pollution_paris_no2_daily_clean.csv")
paris_daily['date'] = pd.to_datetime(paris_daily['date'])

# Charger les donneurs
donors_daily = pd.read_csv(DATA / "no2_all_stations_daily_clean.csv")
donors_daily['date'] = pd.to_datetime(donors_daily['date'])

print(f"\n📁 Données chargées :")
print(f"  • Grenoble : {len(grenoble_daily)} observations, {grenoble_daily['station_id'].nunique()} stations")
print(f"  • Paris : {len(paris_daily)} observations, {paris_daily['station_id'].nunique()} stations")
print(f"  • Donneurs : {len(donors_daily)} observations, {donors_daily['station_id'].nunique()} stations")


📊 STATISTIQUES DESCRIPTIVES - IMPACT DES ZFE SUR LE NO₂

🎯 Dates clés :
  • ZFE Grenoble : 2019-05-02
  • ZFE Paris : 2021-06-01

📁 Données chargées :
  • Grenoble : 5836 observations, 2 stations
  • Paris : 5268 observations, 2 stations
  • Donneurs : 28678 observations, 10 stations


In [ ]:
# ============================================================================
# 1. CARTOGRAPHIE - VISUALISATION GÉOGRAPHIQUE DES ZFE ET STATIONS
# ============================================================================

print("\n" + "=" * 80)
print("1️⃣ CARTOGRAPHIE AVEC GEOPANDAS")
print("=" * 80)

# Charger les périmètres ZFE
with open(DATA / "aires.geojson", encoding="utf-8") as f:
    zfe_geojson = json.load(f)

# Créer GeoDataFrame des ZFE
zfe_features = []
for feat in zfe_geojson['features']:
    pub = feat.get('publisher', {})
    zfe_id = pub.get('zfe_id')
    if zfe_id in ['GRENOBLE', 'PARIS']:
        zfe_features.append({
            'zfe_id': zfe_id,
            'nom': pub.get('nom'),
            'geometry': shape(feat['geometry'])
        })

gdf_zfe = gpd.GeoDataFrame(zfe_features, crs="EPSG:4326")

# Créer GeoDataFrame des stations
def create_stations_gdf(df, zfe_name):
    stations = df[['station_id', 'station_name', 'station_env', 'station_influence', 'lat', 'lon']].drop_duplicates()
    stations['geometry'] = stations.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    stations['ville'] = zfe_name
    return gpd.GeoDataFrame(stations, crs="EPSG:4326")

gdf_grenoble = create_stations_gdf(grenoble_daily, 'Grenoble')
gdf_paris = create_stations_gdf(paris_daily, 'Paris')
gdf_stations = pd.concat([gdf_grenoble, gdf_paris], ignore_index=True)

# Créer une carte pour chaque ville
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

for idx, (ville, ax) in enumerate(zip(['GRENOBLE', 'PARIS'], axes)):
    # Périmètre ZFE
    zfe_subset = gdf_zfe[gdf_zfe['zfe_id'] == ville]
    zfe_subset.plot(ax=ax, color='red', alpha=0.2, edgecolor='red', linewidth=2, label='Périmètre ZFE')
    
    # Stations
    stations_subset = gdf_stations[gdf_stations['ville'] == ville.capitalize()]
    stations_subset.plot(ax=ax, color='blue', markersize=200, alpha=0.7, 
                         edgecolor='darkblue', linewidth=1.5, label='Stations de mesure')
    
    # Annotations
    for _, station in stations_subset.iterrows():
        ax.annotate(station['station_name'], 
                   xy=(station.geometry.x, station.geometry.y),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=9, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    ax.set_title(f"🗺️ ZFE {ville.capitalize()} - Localisation des stations", fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.set_xlabel('Longitude', fontsize=10)
    ax.set_ylabel('Latitude', fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(DATA / 'stats_desc_carte_zfe.png', dpi=300, bbox_inches='tight')
print("✅ Carte sauvegardée : stats_desc_carte_zfe.png")
plt.show()


1️⃣ CARTOGRAPHIE AVEC GEOPANDAS


: 